## 1. Import des bibliotheques

In [2]:
# Bibliothèques standards
import numpy as np
import pandas as pd

# Importation des jeux de données
from sklearn.datasets import fetch_california_housing, load_iris

# Séparation des données (train/test)
from sklearn.model_selection import train_test_split

# Prétraitement des données
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Pipelines
from sklearn.pipeline import Pipeline, make_pipeline

# Modèles de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

# Évaluation et validation des modèles
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

## 2. Chargement et préparation des données

In [3]:
# Chargement du jeu de données Iris
iris = load_iris(as_frame=True)
df_iris=iris.frame
X=df_iris.drop("target", axis=1)
y=df_iris["target"]


## 3. Séparation des données en ensembles d'entraînement et de test

In [4]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Définition du prétraitement des données

In [5]:
# Définir le prétraitement des variables
numeric_features=X.select_dtypes(exclude="object").columns.tolist()
numeric_transformer = StandardScaler()

categorical_features=X.select_dtypes(include="object").columns.tolist()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

## 5. Fonction d'évaluation des modèles

In [6]:
# Fonction pour évaluer les models sur les données test et enregistrer les résultats
def evaluate_model(model_name, pipeline, param_grid):
    grid_search = GridSearchCV(pipeline, param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_estimator = grid_search.best_estimator_
    best_params=grid_search.best_params_

    # Prédire sur les données de test
    y_pred = best_estimator.predict(X_test)
    test_score = accuracy_score(y_test, y_pred)

    results.append({
        'model': model_name,
        'best_estimator': best_estimator,
        'best_score': test_score,
        'best_params': best_params
    })

    print(f"Model: {model_name}, Test Score (Accuracy): {test_score}, Best Params: {grid_search.best_params_}")

## 6. Définition des modèles et des grilles d'hyperparamètres

In [7]:
# Modèles et leurs grilles de paramètres ajustées à tester
models = [
    {
        'name':  'Logistic Regression',
        'pipeline': Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression(max_iter=5000))]),
        'param_grid': {
            'classifier__C' : [0.1, 1, 10]
        }
    },
    {
        'name': 'KNN',
        'pipeline': Pipeline(steps=[('preprocessor', preprocessor), ('classifier', KNeighborsClassifier())]),
        'param_grid': {
            'classifier__n_neighbors': [3, 5, 7]
        }
    },
    {
        'name': 'SVM',
        'pipeline': Pipeline(steps=[('preprocessor', preprocessor), ('classifier', SVC(probability=True))]),
        'param_grid': {
            'classifier__C': [0.1, 1, 10], 'classifier__kernel':['linear', 'rbf']
        }
    },
    {
        'name': 'Random Forest ',
        'pipeline': Pipeline(steps=[('preprocessor', preprocessor), ('classifier', RandomForestClassifier(random_state=42))]),
        'param_grid': { 
            'classifier__n_estimators': [10, 50, 100], 
            'classifier__max_depth': [5, 10, 20],
            'classifier__max_samples': [0.5, 1.0],
            'classifier__max_features': [0.5, 1.0]
        }
    },
     {Ò
        'name': 'Stacking Classifier',
        'pipeline': Pipeline(steps=[
            ('preprocessor', preprocessor), 
            ('classifier', StackingClassifier(
                estimators=[
                    ('logreg', LogisticRegression(max_iter=5000)),
                    ('knn', KNeighborsClassifier()),
                    ('svm', SVC(probability=True)),
                    ('rf', RandomForestClassifier(random_state=42))
                ],
                final_estimator=LogisticRegression(max_iter=5000),
                cv=5,
                n_jobs=-1
            ))
        ]),
        'param_grid': {}  # Pas de paramètres à ajuster ici (par défaut)
    }

]




## 7. Entraînement et évaluation des modèles

In [8]:
# Liste pour stocker les résultats des modèles
results = []
# Évaluation des modèles
for model in models:
    evaluate_model(model['name'], model['pipeline'], model['param_grid'])

Model: Logistic Regression, Test Score (Accuracy): 1.0, Best Params: {'classifier__C': 1}
Model: KNN, Test Score (Accuracy): 1.0, Best Params: {'classifier__n_neighbors': 7}
Model: SVM, Test Score (Accuracy): 1.0, Best Params: {'classifier__C': 0.1, 'classifier__kernel': 'linear'}
Model: Random Forest , Test Score (Accuracy): 1.0, Best Params: {'classifier__max_depth': 5, 'classifier__max_features': 0.5, 'classifier__max_samples': 0.5, 'classifier__n_estimators': 50}
Model: Stacking Classifier, Test Score (Accuracy): 1.0, Best Params: {}


## 8. Sélection et affichage du meilleur modèle

In [9]:
# Afficher les résultats des meilleurs modèles
best_model = min(results, key=lambda x: x['best_score'])
print("\nBest Model:")
print(f"Model: {best_model['model']}, Test Score (MSE): {best_model['best_score']}, Best Params: {best_model['best_params']}")


Best Model:
Model: Logistic Regression, Test Score (MSE): 1.0, Best Params: {'classifier__C': 1}


## 9. Sauvegarde et rechargement du meilleur modèle

In [ ]:
# Enregistrer le meilleur modèle
joblib.dump(best_model['best_estimator'], 'meilleur_modele.pkl')
# Charger le modèle enregistré
meilleur_modele = joblib.load('meilleur_modele.pkl')

# Prédire avec le modèle chargé
predictions = meilleur_modele.predict(X_test)